In [1]:
import pandas as pd

In [2]:
percent_stats = pd.read_csv("../Data/percent_stats.csv")

In [3]:
# Define the selected features for the percentage model
percent_selected_features = ['TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', '3P%', '2P%', 'FT%']

#Define X and y sets
X = percent_stats[percent_selected_features]
y = percent_stats["Pos"]

# Split data into training and testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

#Print the X_train dataframe
X_train.head()

,TRB%,AST%,STL%,BLK%,TOV%,USG%,3P%,2P%,FT%
11173,11.0,7.7,2.0,0.9,10.4,13.1,0.375,0.514,0.827
9174,5.7,15.1,2.9,0.3,16.6,22.0,0.154,0.373,0.724
3412,14.2,4.6,1.8,1.4,12.8,22.5,0.000,0.513,0.813
5939,2.7,22.3,0.0,0.0,63.6,23.4,0.500,0.500,0.000
4719,3.3,37.0,0.0,0.0,33.3,7.5,1.000,0.000,0.000


In [4]:
from sklearn.preprocessing import StandardScaler

# Create a StandardScaler model and fit it to the training data
X_scaler = StandardScaler().fit(X_train)

In [5]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [6]:
X_train_scaled.shape

(12475, 9)

In [7]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

# One-hot encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [8]:
from tensorflow.keras.models import Sequential

model = Sequential()

In [9]:
from tensorflow.keras.layers import Dense
model.add(Dense(units=100, activation='relu', input_dim=9))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=5, activation='softmax'))

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               1000      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 505       
Total params: 21,705
Trainable params: 21,705
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [12]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)

Epoch 1/100
390/390 - 1s - loss: 0.9400 - accuracy: 0.5953
Epoch 2/100
390/390 - 1s - loss: 0.8537 - accuracy: 0.6261
Epoch 3/100
390/390 - 1s - loss: 0.8327 - accuracy: 0.6342
Epoch 4/100
390/390 - 1s - loss: 0.8192 - accuracy: 0.6373
Epoch 5/100
390/390 - 1s - loss: 0.8127 - accuracy: 0.6413
Epoch 6/100
390/390 - 1s - loss: 0.8054 - accuracy: 0.6446
Epoch 7/100
390/390 - 1s - loss: 0.7976 - accuracy: 0.6479
Epoch 8/100
390/390 - 1s - loss: 0.7918 - accuracy: 0.6489
Epoch 9/100
390/390 - 1s - loss: 0.7866 - accuracy: 0.6523
Epoch 10/100
390/390 - 1s - loss: 0.7804 - accuracy: 0.6591
Epoch 11/100
390/390 - 1s - loss: 0.7780 - accuracy: 0.6552
Epoch 12/100
390/390 - 1s - loss: 0.7718 - accuracy: 0.6603
Epoch 13/100
390/390 - 1s - loss: 0.7679 - accuracy: 0.6633
Epoch 14/100
390/390 - 1s - loss: 0.7621 - accuracy: 0.6637
Epoch 15/100
390/390 - 1s - loss: 0.7560 - accuracy: 0.6699
Epoch 16/100
390/390 - 1s - loss: 0.7517 - accuracy: 0.6680
Epoch 17/100
390/390 - 1s - loss: 0.7463 - accura

In [13]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

130/130 - 1s - loss: 2.1331 - accuracy: 0.5597
Normal Neural Network - Loss: 2.1331021785736084, Accuracy: 0.559749960899353


In [14]:
encoded_predictions = model.predict_classes(X_test_scaled[:10])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

/Users/Drew/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [15]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:10])}")

Predicted classes: ['SF' 'PG' 'PF' 'C' 'PG' 'SF' 'SF' 'PG' 'SG' 'PF']
Actual Labels: ['PF', 'PG', 'PF', 'C', 'PG', 'SF', 'PF', 'PG', 'SF', 'C']
